In [2]:
import pandas as pd
import numpy as np
import gensim
import utils
import warnings
%matplotlib inline
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
import seaborn as sns
import xgboost as xgb
import multiprocessing
from gensim.models import FastText
from sklearn import preprocessing
from gensim.models import Doc2Vec
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from collections import defaultdict  # For word frequency
from sklearn.model_selection import train_test_split
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
path='C:\\Users\\jyoti\\Desktop\\NLP\\NLP_project\\NLP-project-\\'
df=pd.read_csv(path + 'Dataset\\3.csv')

In [4]:
df

,Unnamed: 0,caption,following,likes,posts_count,followers,Hod,mon,tue,wed,...,sun,TextBlob_Subjectivity,TextBlob_Polarity,hashtags,hashcounts,mentions,mention_count,emoji,emoji_text,emoji_count
0,0,hand dived isle jura scallop smoked roe leek c...,308.0,100.0,343.0,4202.0,18,0,0,0,...,0,0.000000,0.000000,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,NaN,0,[],NaN,0
1,1,jerusalem artichoke prepared first dessert mal...,308.0,81.0,343.0,4202.0,21,1,0,0,...,0,0.333333,0.250000,"'#michelin', '#michelinstar', '#michelinuk', '...",28,NaN,0,[],NaN,0
2,2,main course seasonal 7 course menu borders roe...,308.0,146.0,343.0,4202.0,19,0,0,1,...,0,0.333333,0.166667,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],NaN,0
3,3,north sea halibut sea vegetables royal beluski...,308.0,163.0,343.0,4202.0,13,1,0,0,...,0,0.000000,0.000000,"'#michelin', '#michelinstar', '#michelinuk', '...",30,NaN,0,[],NaN,0
4,4,adding final touches welcoming guests tonight ...,308.0,251.0,343.0,4202.0,17,0,0,0,...,0,0.791667,-0.041667,NaN,0,"'@_mark_donald', '@theglenturretlalique', '@ca...",11,[],NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26795,26795,alongside hospitality performing arts also mas...,599.0,4079.0,1933.0,167394.0,18,0,0,0,...,0,0.506629,0.428030,"'#lookingforward', '#theatreland', '#performin...",6,"'@evening', '@tiktok', '@nationalyouththeatre'...",4,[],NaN,0
26796,26796,isnt looking forward waking beautiful nights s...,599.0,1621.0,1933.0,167394.0,15,0,0,1,...,0,0.416667,0.491667,"'#lookingforward', '#travelinspiration', '#lov...",7,'@youmustgoblog',1,['🍾'],'bottle with popping cork',1
26797,26797,missing london town heres shot really showcase...,599.0,1548.0,1933.0,167394.0,13,1,0,0,...,0,0.510417,0.260417,"'#lookingforward', '#londonviews', '#prettycit...",8,'@damienhewetson',1,[],NaN,0
26798,26798,closed going try keep happy inspired mix beaut...,599.0,2637.0,1933.0,167394.0,14,0,0,1,...,0,0.708333,0.445833,"'#lookingforward', '#travelinspo', '#datenight...",7,NaN,0,[],NaN,0


In [5]:
df.drop(['Unnamed: 0'], axis = 1,inplace=True)

In [6]:
tokenized_caption = df['caption'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramCaption = FastText(tokenized_caption, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramCaption.build_vocab(tokenized_caption, update=True)
modelFastTextSkipGramCaption.train(tokenized_caption, total_examples= len(tokenized_caption), epochs=20)

In [7]:
modelFastTextSkipGramCaption.wv.most_similar("Gastroenteritis")

[('gastronomic', 0.6996026039123535),
 ('entermake', 0.6905814409255981),
 ('gastronomy', 0.6815277934074402),
 ('gastro', 0.6748406291007996),
 ('aperitivo', 0.6585226058959961),
 ('entering', 0.6276012659072876),
 ('376', 0.6177184581756592),
 ('clientele', 0.6175322532653809),
 ('enter1', 0.6172761917114258),
 ('jr', 0.6171858310699463)]

In [8]:
modelFastTextSkipGramCaption.wv.most_similar("pizza")

[('pizzazz', 0.9135521054267883),
 ('pizzaioli', 0.8163019418716431),
 ('prepizza', 0.7882035970687866),
 ('pizzas', 0.7744265794754028),
 ('pizzavegan', 0.769181489944458),
 ('pizzapoints', 0.7405599355697632),
 ('atpizzatogo', 0.6925336122512817),
 ('pizzathemed', 0.6884173154830933),
 ('iapizzaria', 0.6826822757720947),
 ('piazza', 0.6707425713539124)]

In [9]:
modelFastTextSkipGramCaption.save(path+ "Save Models\\200_fastText_Skipgram_Caption.model")

In [10]:
def word_vector(tokens, size,model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [11]:
wordvec_arrays = np.zeros((len(tokenized_caption), 200)) 
for i in range(len(tokenized_caption)):
    wordvec_arrays[i,:] = word_vector(tokenized_caption[i], 200,modelFastTextSkipGramCaption)
wordvec_caption = pd.DataFrame(wordvec_arrays)
wordvec_caption.shape

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(26800, 200)

## Hashtags

In [12]:
tokenized_hastags = df['hashtags'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramHastags = FastText(tokenized_hastags, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramHastags.build_vocab(tokenized_hastags, update=True)
modelFastTextSkipGramHastags.train(tokenized_hastags, total_examples= len(tokenized_hastags), epochs=20)

In [13]:
modelFastTextSkipGramHastags.wv.most_similar("Gastroenteritis")

[("'#quintessentiallybritish',", 0.7742862105369568),
 ("'#quintessentiallybritish'", 0.7583129405975342),
 ("'#gastroart',", 0.7437229156494141),
 ("'#3aarosettes'", 0.7350507974624634),
 ("'#greatbritishmenu',", 0.7326750755310059),
 ("'#oreechelsea',", 0.7157058715820312),
 ("'#goringdiningroom',", 0.711565375328064),
 ("'#mystorywithmichelin',", 0.7097371816635132),
 ("'#artsandcrafts',", 0.7094197273254395),
 ("'#artdecointerior',", 0.706856906414032)]

In [14]:
modelFastTextSkipGramHastags.save(path+ "Save Models\\200_fastText_Skipgram_Hashtags.model")

In [15]:
wordvec_arrays = np.zeros((len(tokenized_hastags), 200)) 
for i in range(len(tokenized_hastags)):
    wordvec_arrays[i,:] = word_vector(tokenized_hastags[i], 200,modelFastTextSkipGramHastags)
wordvec_Hashtag = pd.DataFrame(wordvec_arrays)
wordvec_Hashtag.shape

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(26800, 200)

## mentions

In [16]:
tokenized_mentions = df['mentions'].apply(lambda x: str(x).split()) # tokenizing
modelFastTextSkipGramMentions = FastText(tokenized_hastags, 
                     size=200, # desired no. of features/independent variables
                     window=2,  # context window size
                     min_count=2, # Ignores all words with total frequency lower than 2.  
                     workers=32, # no.of cores
                     hs = 0,
                     negative = 10, # for negative sampling
                     sg=1  # 1 for Skipgram model
                     )

modelFastTextSkipGramMentions.build_vocab(tokenized_hastags, update=True)
modelFastTextSkipGramMentions.train(tokenized_hastags, total_examples= len(tokenized_hastags), epochs=20)

In [17]:
modelFastTextSkipGramMentions.wv.most_similar("Gastroenteritis")

[("'#quintessentiallybritish',", 0.7962211966514587),
 ("'#greatbritishmenu',", 0.7830856442451477),
 ("'#quintessentiallybritish'", 0.7796211242675781),
 ("'#gastroart',", 0.7580350637435913),
 ("'#3aarosettes'", 0.7443541884422302),
 ("'#goringdiningroom',", 0.7313838005065918),
 ("'#aaronpatterson',", 0.727745771408081),
 ("'#impeccablyenglish',", 0.721266508102417),
 ("'#greentea',", 0.7212579846382141),
 ("'#artsandcrafts',", 0.7187039852142334)]

In [18]:
modelFastTextSkipGramMentions.save(path+ "Save Models\\200_fastText_Skipgram_Hashtags.model")

In [19]:
wordvec_arrays = np.zeros((len(tokenized_hastags), 200)) 
for i in range(len(tokenized_hastags)):
    wordvec_arrays[i,:] = word_vector(tokenized_hastags[i], 200,modelFastTextSkipGramMentions)
wordvec_Mentions = pd.DataFrame(wordvec_arrays)
wordvec_Mentions.shape

C:\Users\jyoti\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(26800, 200)

In [20]:
fastText_Df = pd.DataFrame(np.hstack((wordvec_caption,wordvec_Hashtag,wordvec_Mentions, df)))

In [22]:
fastText_Df

,0,1,2,3,4,5,6,7,8,9,...,612,613,614,615,616,617,618,619,620,621
0,-0.198628,0.238107,0.0516019,-0.224291,-0.153372,-0.49155,-0.183324,-0.173692,-0.503392,0.0680206,...,0,0,0,"'#edinburghfoodies', '#edinburghrestaurants', ...",25,NaN,0,[],NaN,0
1,0.0300783,0.283265,-0.00893267,-0.29003,-0.141337,-0.402387,-0.135627,-0.101424,-0.130859,0.292062,...,0,0.333333,0.25,"'#michelin', '#michelinstar', '#michelinuk', '...",28,NaN,0,[],NaN,0
2,-0.158876,0.308528,-0.047109,0.128602,-0.169392,-0.375455,-0.153029,0.155591,-0.249067,0.17226,...,0,0.333333,0.166667,"'#michelin', '#michelinstar', '#michelinuk', '...",28,"'@castlegamescotland', '@wellocksfood'",2,[],NaN,0
3,-0.00129667,0.457547,-0.117995,-0.147106,-0.0391699,-0.152896,0.07595,0.0752044,-0.294867,-0.00443078,...,0,0,0,"'#michelin', '#michelinstar', '#michelinuk', '...",30,NaN,0,[],NaN,0
4,-0.00347264,0.354553,-0.149874,0.0703828,-0.0584945,-0.0270878,0.0745262,0.038974,-0.401405,-0.0580766,...,0,0.791667,-0.0416667,NaN,0,"'@_mark_donald', '@theglenturretlalique', '@ca...",11,[],NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26795,-0.0490614,0.289419,0.0651602,-0.0125814,-0.140303,0.157489,-0.0449446,0.0496853,-0.308304,-0.0620098,...,0,0.506629,0.42803,"'#lookingforward', '#theatreland', '#performin...",6,"'@evening', '@tiktok', '@nationalyouththeatre'...",4,[],NaN,0
26796,-0.0105777,0.409128,0.0162738,-0.0426455,-0.217991,0.0662325,-0.105371,-0.0542526,-0.390908,-0.151262,...,0,0.416667,0.491667,"'#lookingforward', '#travelinspiration', '#lov...",7,'@youmustgoblog',1,['🍾'],'bottle with popping cork',1
26797,0.0539069,0.271353,0.192338,-0.0256968,-0.154705,0.0279192,-0.011818,-0.0292824,-0.369326,-0.129958,...,0,0.510417,0.260417,"'#lookingforward', '#londonviews', '#prettycit...",8,'@damienhewetson',1,[],NaN,0
26798,-0.1465,0.242222,0.170213,-0.082566,-0.111673,-0.0259936,-0.000669446,-0.0316615,-0.330125,-0.0881454,...,0,0.708333,0.445833,"'#lookingforward', '#travelinspo', '#datenight...",7,NaN,0,[],NaN,0


In [23]:
dropped = [600,615,617,619,620]
fastText_df = fastText_Df.drop(dropped,axis=1)
fastText_df

,0,1,2,3,4,5,6,7,8,9,...,608,609,610,611,612,613,614,616,618,621
0,-0.198628,0.238107,0.0516019,-0.224291,-0.153372,-0.49155,-0.183324,-0.173692,-0.503392,0.0680206,...,0,0,0,1,0,0,0,25,0,0
1,0.0300783,0.283265,-0.00893267,-0.29003,-0.141337,-0.402387,-0.135627,-0.101424,-0.130859,0.292062,...,0,0,0,0,0,0.333333,0.25,28,0,0
2,-0.158876,0.308528,-0.047109,0.128602,-0.169392,-0.375455,-0.153029,0.155591,-0.249067,0.17226,...,1,0,0,0,0,0.333333,0.166667,28,2,0
3,-0.00129667,0.457547,-0.117995,-0.147106,-0.0391699,-0.152896,0.07595,0.0752044,-0.294867,-0.00443078,...,0,0,0,0,0,0,0,30,0,0
4,-0.00347264,0.354553,-0.149874,0.0703828,-0.0584945,-0.0270878,0.0745262,0.038974,-0.401405,-0.0580766,...,0,0,1,0,0,0.791667,-0.0416667,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26795,-0.0490614,0.289419,0.0651602,-0.0125814,-0.140303,0.157489,-0.0449446,0.0496853,-0.308304,-0.0620098,...,0,0,1,0,0,0.506629,0.42803,6,4,0
26796,-0.0105777,0.409128,0.0162738,-0.0426455,-0.217991,0.0662325,-0.105371,-0.0542526,-0.390908,-0.151262,...,1,0,0,0,0,0.416667,0.491667,7,1,1
26797,0.0539069,0.271353,0.192338,-0.0256968,-0.154705,0.0279192,-0.011818,-0.0292824,-0.369326,-0.129958,...,0,0,0,0,0,0.510417,0.260417,8,1,0
26798,-0.1465,0.242222,0.170213,-0.082566,-0.111673,-0.0259936,-0.000669446,-0.0316615,-0.330125,-0.0881454,...,1,0,0,0,0,0.708333,0.445833,7,0,0


In [24]:
FastText_df = pd.DataFrame(fastText_df)
data=FastText_df.astype(str)

normalizer = preprocessing.MinMaxScaler(feature_range=(0, 1))
fastText_df_final = pd.DataFrame(normalizer.fit_transform(data),  columns = data.columns)

In [ ]:
y=fastText_df_final.iloc[ :, -1:]  #Target Variable          
X=fastText_df_final.iloc[: , :-1]  #Feature Matrix 

X=pd.DataFrame(X)
y=pd.DataFrame(y)
for i in X.columns:
  X[i]=pd.to_numeric(X[i])
for i in y.columns:
  y[i]=pd.to_numeric(y[i])


x_train_fastText_df, x_test_fasttext_df, y_train_fastText_df, y_test_fasttext_df = train_test_split(X, y, test_size=0.2,random_state=2022)

In [ ]:
# utils.hyperparameterTuning_XGBoost(x_train_w2v_df,y_train_w2v_df)
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.7, learning_rate = 0.03,max_depth = 5, min_child_weight=4,nthread = 4, n_estimators = 500,subsample=0.7)
xg_reg.fit(x_train_fastText_df, y_train_fastText_df)
y_pred = xg_reg.predict(x_test_fasttext_df)
utils.plot_feature_importance(xg_reg.feature_importances_,x_train_fastText_df.columns,'XG BOOST')

In [ ]:
utils.metrics(y_test_fasttext_df,y_pred)

In [ ]:
# utils.hyperparameterTuning_RandomForest(x_train_w2v_df,y_train_w2v_df)
rnd_clf = RandomForestRegressor(n_estimators=20, min_samples_split=10, min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=True)
rnd_clf.fit(x_train_fastText_df, y_train_fastText_df)
y_pred_fastText_df = rnd_clf.predict(x_test_fasttext_df)
utils.metrics(y_test_fasttext_df,y_pred_fastText_df)
utils.plot_feature_importance(rnd_clf.feature_importances_,x_train_fastText_df.columns,'RANDOM FOREST')